In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Newmark-Beta Sabit Ortalama İvme Yöntemi
def calculate_response(T, m, ksi, dt, acc_file):
    k = 4 * np.pi ** 2 * m / (T ** 2)
    c = 2 * ksi * (k * m) ** 0.5
    acc = np.loadtxt(acc_file, skiprows=4)
    rows = acc.shape[0]
    cols = acc.shape[1]
    npts = rows * cols
    acc = acc.reshape(npts)
    
    # Başlangıç Koşulları
    p = -m * acc
    u0 = 0
    ud0 = 0
    gamma = 0.5
    beta = 0.25
    udd0 = (p[0] - c * ud0 - k * u0) / m
    a1 = (1 / (beta * dt ** 2) * m) + gamma / (beta * dt) * c
    a2 = (1 / (beta * dt)) * m + (gamma / beta - 1) * c
    a3 = (1 / (2 * beta) - 1) * m + dt * (gamma / (2 * beta) - 1) * c
    kh = k + a1

    # Artırımsal Zamana Bağlı Çözüm
    u = np.zeros(npts)
    ud = np.zeros(npts)
    udd = np.zeros(npts)
    u[0] = u0
    ud[0] = ud0
    udd[0] = udd0
    for i in range(0, npts - 1):
        ph_i_plus_1 = p[i + 1] + a1 * u[i] + a2 * ud[i] + a3 * udd[i]
        
        u[i + 1] = ph_i_plus_1 / kh
        ud[i + 1] = (gamma / (beta * dt)) * (u[i + 1] - u[i]) + (1 - gamma / beta) * ud[i] + dt * (1 - gamma / (2 * beta)) * udd[i]
        udd[i + 1] = (1 / (beta * (dt ** 2))) * (u[i + 1] - u[i]) - (1 / (beta * dt)) * ud[i] - (1 / (2 * beta) - 1) * udd[i]    

    # Grafikler
    t = np.arange(0, npts * dt, dt)
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    # Yer Değiştirme
    axs[0].plot(t, u, color="red")
    axs[0].set_xlabel("Zaman (s)")
    axs[0].set_ylabel("Yer Değiştirme (m)")
    max_u = np.max(u)
    max_u_index = np.argmax(u)
    axs[0].plot(t[max_u_index], max_u, 'ro')
    axs[0].annotate(f'Max: {max_u:.2f}', xy=(t[max_u_index], max_u), xytext=(t[max_u_index] + dt, max_u))

    # Hız
    axs[1].plot(t, ud, color="blue")
    axs[1].set_xlabel("Zaman (s)")
    axs[1].set_ylabel("Hız (m/s)")
    max_ud = np.max(ud)
    max_ud_index = np.argmax(ud)
    axs[1].plot(t[max_ud_index], max_ud, 'bo')
    axs[1].annotate(f'Max: {max_ud:.2f}', xy=(t[max_ud_index], max_ud), xytext=(t[max_ud_index] + dt, max_ud))

    # İvme
    axs[2].plot(t, udd, color="green")
    axs[2].set_xlabel("Zaman (s)")
    axs[2].set_ylabel("İvme (m/s^2)")
    max_udd = np.max(udd)
    max_udd_index = np.argmax(udd)
    axs[2].plot(t[max_udd_index], max_udd, 'go')
    axs[2].annotate(f'Max: {max_udd:.2f}', xy=(t[max_udd_index], max_udd), xytext=(t[max_udd_index] + dt, max_udd))

    plt.tight_layout()
    plt.show()

